In [ ]:
# 测试模型运行
from transformers import AutoModelForCausalLM, AutoTokenizer

model_name = "/root/.cache/huggingface/hub/models--Qwen--Qwen2.5-7B-Instruct/snapshots/a09a35458c702b33eeacc393d103063234e8bc28" # Qwen/Qwen2.5-7B-Instruct

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)


In [ ]:

instruction = """你是一个文本实体识别领域的医学专家，你需要从给定的句子中提取中医诊断,中药,中医治疗, 方剂, 西医治疗, 西医诊断 '其他治疗'等. 以 json 格式输出, 如 {'口苦': '临床表现','肺结核': '西医诊断'} 注意: 1. 输出的每一行都必须是正确的json字符串. 2.找不到任何实体时, 输出"没有找到任何实体"."""
prompt = "其临床表现有烧心、反酸、胸骨后灼痛等症状，自述存在功能性消化不良"
messages = [
    {"role": "system", "content": instruction},
    {"role": "user", "content": prompt}
]
text = tokenizer.apply_chat_template(
    messages,
    tokenize=False,
    add_generation_prompt=True
)
model_inputs = tokenizer([text], return_tensors="pt").to(model.device)

generated_ids = model.generate(
    **model_inputs,
    max_new_tokens=512
)
generated_ids = [
    output_ids[len(input_ids):] for input_ids, output_ids in zip(model_inputs.input_ids, generated_ids)
]

response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
response